# Lesson 19: Graph coloring

## Overview 

__Summary:__ One of the most powerful and far-reaching applications of graph theory has to do with _scheduling problems_ and their solutions. We've already seen an instance of this with the "committee graph" in [Lesson 14](http://nbviewer.jupyter.org/github/RobertTalbert/discretecs/blob/master/lessons/lesson14.ipynb), where we created a graph to show the connections between committees that could not meet at the same time and used a method of coloring the nodes to indicate the time slots needed. In this lesson we take up the issue of graph coloring in more depth. We will learn about _proper_ and _optimal vertex colorings_ of graphs, the _chromatic number_ of a graph, and move toward some theorems that place bounds on the chromatic numbers of graphs. We will also look at a "greedy algorithm" approach to finding proper colorings of graphs and set the stage for using graph coloring to solve more applied problems in the same vein as the committee problem. 

This lesson addresses the following learning targets: 

+ __GT.5:__ I can give a valid vertex coloring for a graph and determine the chromatic number of the graph.



## Background

Back in Lesson 14, when we first introduced graphs, we used a graph to model a collection of committees and to solve the problem of finding time slots when the committees could meet without conflicts. We based the graph on the committee members, namely: 

+ The nodes of the graph were the committees themselves, and 
+ If two committees have a person in common, then put an edge incident with the nodes that represent those committees. 

The committees in that example were: 

| Committee | Members | 
|:--------- | :------ |
| $c1$     | 1, 2, 3 |
| $c2$     | 1, 4, 5, 8 | 
| $c3$     | 2, 3, 7 | 
| $c4$     | 3, 5, 10 | 
| $c5$     | 4, 10, 11, 12 | 
| $c6$     | 1, 6, 9, 11 | 
| $c7$     | 6, 11, 12 | 
| $c8$     | 2, 9, 11, 12 |

And the graph was: 

<img src="committees.png" height="300" width="300"> 

You were asked whether all eight committees could be scheduled into four one-hour blocks without any conflicts. The answer turned out to be "yes" and there was more than one way to do it. A way to think about solving this problem is the following: Use colors to represent the time slots. Color two nodes differently if they cannot meet in the same time slot -- that is, __color the nodes differently if they have an edge connecting them__. 

Here is one solution to the problem, showing the committee graphs with the nodes colored correctly: 

<img src="committee_color.png"> 

So we see from the colored graph that we can schedule c1 and c5 together, c2 and c8, c3 and c7, and c4 and c6. Four time slots is sufficient. 

However this raises some more questions: 

+ We found a way to schedule all the committees using _four_ time slots. Can we do it in three? In two? __What is the smallest number of colors needed to color this graph in the way we are describing, so that no two adjacent nodes have the same color?__ 
+ __How did we come up with this coloring in the first place?__ We can do so by trial and error, but this doesn't scale up well. What if we had not 8 committees but 800, for example if we were trying to schedule classes at a university without conflicts, or stage performance times at a rock music festival, etc.? Or even many times more than this, for example if we are writing low-level driver software to schedule CPU cycles in a computer without conflicts? __We need an algorithm.__
+ Finally, a mathematician would ask: __What is the relationship between the number of colors needed to color the nodes of the graph, and the structure of the graph itself?__ For example, if the graph has a lot of nodes, must it use a lot of colors? Or is the relationship deeper and more complicated than this? 

All of these questions are what we are going to address in this lesson. 

---

First, let's define what we mean by a coloring. This should look familiar. 

>__Definition:__ A __proper coloring__ of a graph $G$ is an assignment of colors (which are considered elements of some set, not necessarily real colors) to the nodes of $G$, in such a way that adjacent nodes have different colors. 

This is just the same thing as the coloring scheme we defined with the committee graph: The coloring above is proper because every node has a color and if two nodes are adjacent, their colors are different. 

__Aside:__ What we are defining here is more correctly called a proper __vertex__ coloring of the graph. There is a related notion of an _edge coloring_ of a graph ([click here for more on that](http://mathworld.wolfram.com/EdgeColoring.html)), but since we will only think about vertex colorings in MTH 325 we will drop the word "vertex" and assume from here on out that all colorings refer to vertex colorings. 


Here is another example of a proper coloring of a larger graph: 

<img src="color2.png"> 

Notice that this coloring uses 5 colors. Do you think we can do it in less than 5 colors? 

On the other hand here is an example of a "non-proper" coloring on a small graph: 

<img src="improper_coloring.png"> 

This isn't a proper coloring because we have adjacent nodes that have the same color, for example 3 and 4, or 0 and 5. 



---

The 10-node graph above that is properly colored makes us think about __how we create proper colorings for a graph__ in an algorithmic way that's fast and scalable to large networks. Let's think about this problem for a minute. 

It's clear that one way to do properly color a graph is to just __give every node in the graph a different color__. That way we'd be guaranteed to have adjacent nodes colored differently because _every_ node is colored differently. Here is what the committee graph would look like if we did it that way: 

<img src="comm_colors2.png"> 

This is very pretty, but it leads to practical issues when you think about scheduling committees. What are those practical issues? Stop and think about it. 

So what we really want is not merely to color a graph properly, but __to color the graph properly using the smallest possible number of colors__. A proper coloring that uses the smallest possible number of colors -- so that any coloring with a smaller number of colors would have to be non-proper -- is called an __optimal proper coloring__. 

There's another term that goes with the concept of an optimal proper coloring: 

>__Definition:__ Given a graph $G$, the __chromatic number__ of $G$ is the smallest number of colors needed to create a proper coloring of the graph. That is, it is the number of colors needed for any optimal proper coloring. We denote the chromatic number of $G$ by $\chi(G)$. 

The symbol $\chi$ is the Greek letter "chi" -- pronounced __KIE__, with a hard "K" sound and rhyming with "pie" and "sky". (In LaTeX, you can get this by typing `\chi` in math mode.)  

Here is one more related term: 

>__Definition:__ A graph $G$ is said to be $k$-colorable if a proper coloring of its nodes is possible with $k$ colors. 

Back to the committee graph, which has 8 nodes -- this graph is obviously 8-colorable. But it is also 7-, 6-, 5-, and 4-colorable. We looked at the 4-coloring earlier in the notes. The question now becomes: _Is it 3-colorable?_ In class when discussing Lesson 14, we attempted to create a 3-coloring of this graph and failed. But this doesn't mean we _can't_ color the graph with 3 colors -- maybe we just need to keep trying. Is it possible to 3-color the committee graph, or is there something about this graph that tells us that it will never work no matter how hard we try? 

__Spoilers:__ It turns out we cannot color the committee graph with 3 colors no matter how many different coloring schemes we try. We will think about "why" in our class meeting for this lesson. For now, just know that in symbol language, that would mean that the chromatic number of the committee graph is 4. That is, if $C$ is the committee graph then $\chi(C) = 4$. 

--- 

What's next? 

We're going to leave the background material for this lesson right here. What's next for us is to play with problems that help us understand the three main questions of the lesson stated earlier, which we can reframe now as follows: 

+ How do we find the chromatic number of a graph? 
+ What's an algorithm for coming up with an optimal proper coloring of a graph? 
+ What are some theorems that describe precise relationships between the chromatic number of a graph and the internal structure of the graph -- the number of nodes, number of edges, degrees of nodes, etc? 

## Preview Activities

Found here on Formative: https://goformative.com/student/#/assignments/NXUH974 

Please note that starting with this lesson, preview activities are due __on the day of the lesson__ rather than at the meeting prior to that lesson. This will be reinforced on the calendar and announced with a little more detail via Blackboard. 

Also note the change in the final question. 

## Daily Homework

Starting with this lesson we are going to do Preview Activities and Daily Homework differently than we have done in the past. We will need more time in class to explore the questions asked above, so __Daily Homework from now on will not be checked in class__. Instead, these Daily Homework questions will be used as group work questions you'll do "live" with your table groups when you arrive. The questions are still contained here in the lesson notes, and you are encouraged to work ahead on these to help your group and to give more time in class for deeper work. But they will not be checked. 

Instead, your Daily Homework score for this lesson -- and if it works out, for subsequent lessons as well -- will be entirely based on a Wrap-Up activity given on Formative at the end of class. Consider it to be a participation score, but you have to stay through the end of class and contribute something meaningful to the Wrap-Up activity to get the "Pass" mark for Daily Homework. Your responses on wrap-up activities will still be based entirely on completeness and effort just like usual, so all you have to do is attend the entire class and give good feedback at the end. 

1. [Go back to Lesson 16](http://nbviewer.jupyter.org/github/RobertTalbert/discretecs/blob/master/lessons/lesson16.ipynb) and review the definition of $K_{m,n}$, the _complete bipartite graph_. Recall that Sage can produce $K_{m,n}$ using `graphs.CompleteBipartiteGraph(m,n)`. For example $K_{3,6}$ is produced using `graphs.CompleteBipartiteGraph(3,6)`. Use Sage to determine $\chi(K_{m,n})$ for several values of $m$ and $n$. (Sage can do this using the line `graphs.CompleteBipartiteGraph(m,n).chromatic_number()` with the `m` and `n` changed to specific values.) What do you notice? Why is this happening? 
2. What is the chromatic number of $Q_n$, the cube graph with $2^n$ nodes? Play around with this in Sage by computing the chromatic number of `graphs.CubeGraph(n)` for several values of $n$. If you find a pattern, try to explain why the pattern is happening. 
2. If $G$ is a graph, then $\Delta(G)$ (pronounced "Delta G") is the largest degree of any node in $G$. For example, if $C$ is the committee graph, then $\Delta(C) = 5$ because the largest degree of any node is 5 (and three different nodes have this degree). In Sage, run the following code several times: It generates a random graph $G$, then prints off its chromatic number $\chi(G)$ and its maximum degree $\Delta(G)$ Make notes of your results. Is there a relationship between $\chi(G)$ and $\Delta(G)$? State your observations in the form of a conjecture that is mathematically as precise as possible. 

Code for question 3: 

    g = graphs.RandomGNP(10, 0.5)
    chi = g.chromatic_number()
    delta = max(g.degree_sequence()) # Pulls off the biggest element of g's degree sequence
    print("chi(g) = %d, Delta(g) = %d" % (chi, delta))

Recall that `graphs.RandomGNP(n,p)` generates a graph with $n$ nodes in which the probability of two nodes being adjacent is $p$. So for example `graphs.RandomGNP(10, 0.5)` create a graph with 10 nodes and a 50/50 chance of two nodes being adjacent. Run and re-run this code several times, varying the "10" and "0.5" to try out different numbers of nodes and different probabilities of being connected. 

---

## Appendix: Working with graph coloring in Sage

You probably noticed that the colored graphs above were produced in Sage. The good news is that Sage has awesome capabilities for working with graph coloring. The bad news is that not all of these capabilities are available in Jupyter notebooks. I will try now to explain what you can do, and what you cannot do, using our usual setup of Jupyter notebooks running the SageMath kernel. 

First of all, Sage can determine an optimal coloring of any graph using some special functions that are found in a library that we can load. To load the library, make sure the SageMath kernel is selected and enter: 

In [8]:
from sage.graphs.graph_coloring import *

[Here is the documentation for all the stuff that is contained in this library.](http://doc.sagemath.org/html/en/reference/graphs/sage/graphs/graph_coloring.html) Take some time to check out the examples and play around in a scratch notebook. For our purposes, the two most items in this library are the method `.chromatic_number()` and the function `vertex_coloring()`.  

The `chromatic_number()` method does what you expect: 

In [9]:
# The committee graph: 
c = Graph([('c1', 'c2'), ('c1','c3'), ('c1','c4'), ('c1','c6'), ('c1','c8'),
     ('c2', 'c4'),  ('c2', 'c5'),
     ('c3', 'c4'),  ('c3', 'c8'),
     ('c4', 'c5'),
     ('c5', 'c6'),  ('c5', 'c7'),  ('c5', 'c8'),
     ('c6', 'c7'),  ('c6', 'c8'),
     ('c7', 'c8')])

c.chromatic_number()

4

Another example: Let's see if there's a pattern in the chromatic numbers of the cycle graphs $C_n$: 

In [10]:
for i in range(2, 10):
    print("The cycle graph with %d nodes has chromatic number %d." % (i,graphs.CycleGraph(i).chromatic_number()))

The cycle graph with 2 nodes has chromatic number 2.
The cycle graph with 3 nodes has chromatic number 3.
The cycle graph with 4 nodes has chromatic number 2.
The cycle graph with 5 nodes has chromatic number 3.
The cycle graph with 6 nodes has chromatic number 2.
The cycle graph with 7 nodes has chromatic number 3.
The cycle graph with 8 nodes has chromatic number 2.
The cycle graph with 9 nodes has chromatic number 3.


Hmmmm. Care to make a conjecture? 

The other item mentioned, `vertex_coloring()`, takes a graph and gives a "coloring" of the nodes by grouping nodes of the same color together. The output is a list with lists inside it, each of the interior lists representing a color. 

Note that `vertex_coloring()` is a FUNCTION, not a method. So you plug in the graph to the paretheses -- don't append `vertex_coloring()` with a dot. 

Example: 

In [12]:
# Coloring the committee graph
vertex_coloring(c)

[['c6', 'c4'], ['c5', 'c1'], ['c8', 'c2'], ['c7', 'c3']]

In [13]:
# Coloring the cycle graph C9
vertex_coloring(graphs.CycleGraph(9))

[[8, 6, 4, 2], [7, 5, 3, 1], [0]]

Unfortunately you have to imagine, rather than see, the colors. For example in $C_9$ we can color 2, 4, 6, 8 red; 1, 3, 5, 7 blue; and 0 purple. 

Sage _can_ actually produce a graph with the nodes colored. _But_, this functionality does not work when running the Sage kernel from within a Jupyter notebook. If you really want to use Sage to color a graph, you have two options: 

1. __Use the Sage Cell Server.__ [This is a website that consists of a single Sage cell](https://sagecell.sagemath.org/) (you can change the kernel here to different languages). You can use this website to do "one-off" computations in Sage or Python and avoid having to make a scratch notebook. It's very useful. You can set the kernel here to Sage and then execute the code given below. 
2. __Use a SageMath worksheet on SageMath Cloud.__ You may have noticed that one of the objects you can create in SMC is a "SageMath worksheet". We have never done this in class and we typically do not use these. However certain Sage functions work in these worksheets where they do not work elsewhere. SageMath worksheets predate Jupyter notebooks, and they are not as user-friendly, aesthetically appealing, or functional as Jupyter notebooks -- but you can still make them and use them. Create one on SMC as you would create a Jupyter notebook and then use the code below.  

Here is an example to show how to create the colored graph in Sage. In the first line we are just creating a graph out of an edge list. 

    from sage.graphs.graph_coloring import *   # This isn't necessary if the library was imported already
    example = Graph([(0,1), (0,2), (0,3), (1,2), (2,3), (2,4), (3,4), (2,5)])
    example_colors = vertex_coloring(example)
    example.show(partition=example_colors)
    
In other words: (1) Create the graph, (2) use `vertex_coloring` to create the list of colors, and then (3) pass `partition=` and then the name of your color list as a parameter to the `.show()` method. 